In [1]:
import matplotlib
%matplotlib inline

# need to install skimage. on Mac, use the following command
# pip install scikit-image
import numpy as np
import os
import sys
import math
import glob
from matplotlib import pyplot as plt
import skimage.transform
import skimage.io
import skimage.filters
import skimage.util
import glob
import math
from skimage import color
from numpy import linalg
from PIL import Image
from IPython.display import clear_output
from skimage.measure import regionprops
from skimage import filters
from skimage.color import label2rgb
import skvideo.io # this could be put on the top of the script

# root_folder ='/Users/mandy/Documents/Video_occlusion/'
# folder_name = 'Videos/'

# Helper functions

In [2]:
def Remove(duplicate):
    final_list = []
    for num in duplicate:
        if num not in final_list:
            final_list.append(num)
    return final_list

# scale the mask bigger or smaller
def zoom(image,center,zoom_factor):
    if zoom_factor >= 1:
        s_shape = image.shape
        image_resized = skimage.transform.rescale(image, zoom_factor, preserve_range = True)
        l_shape = image_resized.shape
        crop_width = []
        up_crop = center[0]*abs(1-zoom_factor)
        low_crop = (s_shape[0]-center[0])*abs(1-zoom_factor)
        left_crop = center[1]*abs(1-zoom_factor)
        right_crop = (s_shape[1]-center[1])*abs(1-zoom_factor)
        crop_width.append((up_crop,low_crop))
        crop_width.append((left_crop,right_crop))
        cropped_image = skimage.util.crop(image_resized, crop_width)
    else:
        original_shape = image.shape
        cropped_image = np.zeros(original_shape)
        image_resized = skimage.transform.rescale(image, zoom_factor, preserve_range = True)
        up_crop = int(center[0]*abs(1-zoom_factor))
        left_crop = int(center[1]*abs(1-zoom_factor))
        cropped_image[up_crop:int(up_crop+image_resized.shape[0]),left_crop:int(left_crop+image_resized.shape[1])] = image_resized
    return cropped_image

# apply gaussian blur to the area where the mask is
def applyBlur(image, mask, sigma=50.0, feather=5):
    image = skimage.util.img_as_float(image)
    image = skimage.transform.resize(image,mask.shape, preserve_range = True)
    blurred = skimage.filters.gaussian(image, sigma=sigma, multichannel = True) #skimage.util.img_as_float(mask)*256
    blurredmask = skimage.filters.gaussian(skimage.util.img_as_float(mask), sigma=feather, multichannel=True)
    blurredmask = np.expand_dims(blurredmask, 2)
    inverted_mask =np.ones([blurredmask.shape[0],blurredmask.shape[1],1])-blurredmask
    blended = blurredmask*blurred + inverted_mask*image
    return blended

# apply gaussian blur to the area except the mask
def applyBlurContext(image, mask, sigma=50.0, feather=5):
    image = skimage.util.img_as_float(image)
    image = skimage.transform.resize(image,mask.shape, preserve_range = True)
    blurred = skimage.filters.gaussian(image, sigma=sigma, multichannel=True)
    blurredmask = skimage.filters.gaussian(skimage.util.img_as_float(mask), sigma=feather, multichannel=True)
    blurredmask = np.expand_dims(blurredmask, 2)
    inverted_mask =np.ones([blurredmask.shape[0],blurredmask.shape[1],1])- blurredmask
    blended = inverted_mask*blurred + blurredmask*image
    return blended

# determine input and output parameters

In [3]:
movie_name = 'Forrest_Gump' 
JPEG_DIR = 'FrameImages' # where to read the frames
ANNOTATIONS_DIR = 'Annotations' # where to read the annotated iamges
DICTIONARY_DIR = 'DictOutput' # where to read the masks
BLURRED_DIR = 'BluredImages' # where to save blurred frames
NEWMASKDIR = 'NewMask'# where to read the selected masks
CONTEXT_DIR = 'ContextOccluded'

outdir = os.path.join(BLURRED_DIR, movie_name)
if not os.path.exists(outdir):
    os.makedirs(outdir)
outdir = os.path.join(NEWMASKDIR, movie_name)
if not os.path.exists(outdir):
    os.makedirs(outdir)
outdir = os.path.join(BLURRED_DIR, movie_name, CONTEXT_DIR)
if not os.path.exists(outdir):
    os.makedirs(outdir)

# apply temporal smoothing to make masks look better

In [ ]:
testing_flag = 0
frame_list =  sorted(glob.glob(os.path.join(JPEG_DIR, movie_name,'*.jpg')))
start_index = 0 # start index of frames to read
image_first = plt.imread(frame_list[start_index])
original_shape = image_first.shape
IOU_THRESHOLD = 0.85 # intersection over union threshold
mask_list =  sorted(glob.glob(os.path.join(NEWMASKDIR, movie_name,'*.npz')))
Temporal_mean = 15 # avearge with N frames before and N frames after if they satisfy the constraint; the bigger the smoother
end_index = len(mask_list) # end index of frames to read

zoom_mask_character = 1 # scale the character's mask by this factor; > 1 bigger; <1 smaller
zoom_masks_context = 0.85  #scale the context's mask by this factor; > 1 bigger; <1 smaller
threshold_value = 0.5

for mask_id, mask_path in enumerate(mask_list[start_index:end_index]):
    image_path = mask_list[mask_id+start_index]
    frame_path = frame_list[mask_id+start_index]
    original_frame = plt.imread(frame_path)
    mask_npz = np.load(image_path,encoding = 'latin1',fix_imports = True) # get mask
    mask_file = mask_npz.files # get the mapping name
    new_mask=mask_npz[mask_file[0]] # use the mapping name to retrieve masks
    image_name = image_path[-9:-4]
    
    # determine the masks to average
    mask_selected_index = []
    mask_num = mask_id + start_index
    
    # for mask index before the target mask
    if mask_num <= Temporal_mean:
        for i in np.arange(mask_num):
            mask_selected_index.append(i)
    else:
        for i in np.arange(mask_num-Temporal_mean, mask_num):
            mask_selected_index.append(i)
        
    # for mask index after the target mask
    if (mask_num + Temporal_mean) >= end_index:
        for i in np.arange(mask_num,end_index):
            mask_selected_index.append(i)
    else:
        for i in np.arange(mask_num, mask_num + Temporal_mean):
            mask_selected_index.append(i)
    
    # see if the mask are all similar
    mask_count = 1
    mask_to_average = new_mask
    for idx in mask_selected_index:
        mask_tmp_npz = np.load(mask_list[idx],encoding = 'latin1',fix_imports = True) # get mask
        mask_tmp_file = mask_tmp_npz.files # get the mapping name
        mask_tmp=mask_tmp_npz[mask_tmp_file[0]] # use the mapping name to retrieve masks
        union_area = np.sum((mask_tmp+new_mask)>0) #union area
        join_area = np.sum((mask_tmp+new_mask)>1) #union area
        if union_area > 0:
            if (float(join_area)/float(union_area) > IOU_THRESHOLD):
                mask_count = mask_count + 1
                mask_to_average = mask_to_average + mask_tmp

    averaged_mask = mask_to_average/float(mask_count)
    
    # get the center of mask
#     threshold_value = filters.threshold_otsu(averaged_mask)
    labeled_foreground = (averaged_mask > threshold_value).astype(int)
    properties = regionprops(labeled_foreground,averaged_mask)
    if len(properties)>0:
        weighted_center_of_mass = properties[0].weighted_centroid
    else:
        weighted_center_of_mass = np.round(np.divide(labeled_foreground.shape,2))

    ## blur
    # substitutded averaged_mask as labeled_foreground here
    new_averaged_mask = zoom(labeled_foreground, weighted_center_of_mass, zoom_mask_character)
    blurred_frame = applyBlur(original_frame, new_averaged_mask)
    blurred_frame = skimage.transform.resize(blurred_frame,(original_shape[0],original_shape[1]), preserve_range = True)
    plt.imsave(os.path.join(BLURRED_DIR,movie_name, image_name + '.jpg'), blurred_frame)  

    # save context occluded blurred image
    new_averaged_mask = zoom(labeled_foreground, weighted_center_of_mass, zoom_masks_context)
    blurred_frame_context = applyBlurContext(original_frame, new_averaged_mask)
    blurred_frame_context = skimage.transform.resize(blurred_frame_context,(original_shape[0],original_shape[1]), preserve_range = True)
    plt.imsave(os.path.join(BLURRED_DIR,movie_name,CONTEXT_DIR, image_name + '.jpg'), blurred_frame_context)  

    # report progress
    progress = np.divide(np.float32(mask_id+start_index),np.float32(end_index))
    if np.remainder(mask_id,20)== 0:
        clear_output()
        
    print('Processing '+ 'image ' + str(mask_id) + '/'+ str(end_index-start_index)+ '. Progress =' + str(progress))

# generate new video using the blurred frames

In [ ]:
# video for occluding the target character
fps = '25'
start_index = 0
blurred_frame_list =  sorted(glob.glob(os.path.join(BLURRED_DIR, movie_name,'*.jpg')))
end_index = len(blurred_frame_list)
first_frame = plt.imread(blurred_frame_list[0])
video_shape = (2,first_frame.shape[0],first_frame.shape[1],first_frame.shape[2])
if testing_flag:
    output_name = movie_name + '_test.mp4'
else:
    output_name = movie_name + '.mp4'
vid_out = skvideo.io.FFmpegWriter(output_name,
    outputdict={
      '-vcodec': 'libx264','-pix_fmt': 'yuv420p','-r': fps})
blendedvideo = np.zeros(video_shape)
for frame in blurred_frame_list[start_index:end_index]:
    blurred_image = plt.imread(frame)
    blendedvideo[0,:,:,:] = blurred_image
    vid_out.writeFrame(blendedvideo[0,:,:,:])        
vid_out.close()
print('finished generating character-only videos')

# video for occluding the context
blurred_frame_list =  sorted(glob.glob(os.path.join(BLURRED_DIR, movie_name, CONTEXT_DIR, '*.jpg')))
first_frame = plt.imread(blurred_frame_list[0])
end_index = len(blurred_frame_list)
video_shape = (2,first_frame.shape[0],first_frame.shape[1],first_frame.shape[2])
if testing_flag:
    output_name = movie_name + 'Context_test.mp4'
else:
    output_name = movie_name + 'Context.mp4'
vid_out = skvideo.io.FFmpegWriter(output_name,
    outputdict={
      '-vcodec': 'libx264','-pix_fmt': 'yuv420p','-r': fps})
blendedvideo = np.zeros(video_shape)
for frame in blurred_frame_list[start_index:end_index]:
    blurred_image = plt.imread(frame)
    blendedvideo[0,:,:,:] = blurred_image
    vid_out.writeFrame(blendedvideo[0,:,:,:])        
vid_out.close()
print('finished generating context-only videos')